<h1> NEAT Implementation Practice </h1>
A first-pass attempt at implementing NEAT by translating SethBling's neatevolve.lua script into Python.

In [2]:
import gym
import numpy as np

In [3]:
# first define the environment and get the both the action and observation spaces 
envName = 'CartPole-v0' # just need to change this if you wanna try it in environments
env = gym.make(envName)

# print out the observation and action spaces
print(env.action_space.n) # number of valid actions we can take 
print(env.observation_space.shape[0]) # number of "sensors" of the environment that we have (number of inputs)

# NOTE: Need to keep track of the valid values these can take 
# NOTE: Also need to have inputs + 1, with the 1 being the bias (stc?)
inputs = env.action_space.n
outputs = env.observation_space.shape[0] + 1



2
4


In [4]:
# some NEAT hyperparameters 
# NOTE: Go back to this and change accordingly (remove what you dont need)

population = 300
deltaDisjoint = 2.0
deltaWeights = 0.4
deltaThreshold = 1.0

staleSpecies = 15

mutateConnectionsChance = 0.25
perturbChance = 0.90
crossOverChance = 0.75
linkMutationChance = 2.0
nodeMutationChance = 0.5
biasMutationChance = 0.4
stepSize = 0.1
disableMutationChance = 0.4
enableMutationChance = 0.2

timeoutConstant = 20 

maxNodes = 1000000